In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from matplotlib.projections import projection_registry

import project_fitness_function as fit

In [ ]:
file = 'train_data.csv'
terreno_hectares = 2
limite_agua = 8.5
janela_dias = 90
orcamento_maximo = 7.5

interacoes = 500

In [ ]:
df = pd.read_csv(file)

In [ ]:
m2_area = terreno_hectares * 10_000

In [ ]:
melhor_resultado = []
fitness_history = []
std_fitness_history = []
best_fitness_so_far = []
avg_fitness_history = []
resultados = []

# Seleciona os genes dos 3 melhores indivíduos por geração e cria um novo dataframe
def selecao_genes():
    melhores_amostras = [item['sample'] for item in resultados[:2]]
    return pd.concat(melhores_amostras, ignore_index=True).drop_duplicates()

# Calcula a adaptação das amostras por geracão
def adaptacao(amostras):
    for i, amostra in enumerate(amostras):
        fit_result = fit.project_fitness_function(amostra, orcamento_maximo, m2_area, limite_agua, janela_dias)
        resultados.append({'sample': amostra, 'fitness': fit_result})

    resultados.sort(key=lambda x: x['fitness'],reverse=True)

    fitness_values = [item['fitness'] for item in resultados]
    std_fitness = np.std(fitness_values)

    # Adiciona à lista de desvios padrão
    std_fitness_history.append(std_fitness)
    avg_fitness = sum(item['fitness'] for item in resultados) / len(resultados)

    # Salva informações para gerar gráfico da média por geração
    avg_fitness_history.append(avg_fitness)

    # Salva dados para gerar grafico de convergência do algoritimo
    fitness_history.append(resultados[0]['fitness'])

    if len(melhor_resultado) == 0:
        melhor_resultado.append(resultados[0])

    if resultados[0]['fitness'] > melhor_resultado[0]['fitness']:
        melhor_resultado[0] = resultados[0]

    best_fitness_so_far.append(melhor_resultado[0]['fitness'])


# array que contém população inicial
populacao_inicial = []

# gera 100 amostras e preenche a população inicial
for i in range(100):
    populacao_inicial.append(df.sample(5))

# executa para população inicial
adaptacao(populacao_inicial)

melhores_genes = selecao_genes()

#crossover de genes para a quantidade de interações configuradas
for i in range(interacoes):
    nova_populacao = fit.crossover(populacao_inicial, melhores_genes)
    adaptacao(nova_populacao)
    melhores_genes = selecao_genes()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(range(len(fitness_history)), fitness_history, linestyle='-', color='b')
plt.title('Gráfico de Convergência do Algoritmo Genético')
plt.xlabel('Iteração')
plt.ylabel('Melhor Fitness')
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(range(len(std_fitness_history)), std_fitness_history, linestyle='-', color='m')
plt.title('Gráfico de Desvio Padrão de Aptidão do Algoritmo Genético')
plt.xlabel('Iteração')
plt.ylabel('Desvio Padrão de Fitness')
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(range(len(best_fitness_so_far)), best_fitness_so_far, linestyle='-', color='r')
plt.title('Gráfico de Melhor Aptidão Encontrada até o Momento')
plt.xlabel('Iteração')
plt.ylabel('Melhor Fitness')
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(range(len(avg_fitness_history)), avg_fitness_history, linestyle='-', color='g')
plt.title('Gráfico de Média de Aptidão do Algoritmo Genético')
plt.xlabel('Iteração')
plt.ylabel('Média de Fitness')
plt.grid(True)
plt.show()